In [120]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import csv
from datetime import datetime
from requests.auth import HTTPProxyAuth
import requests

In [1]:
proxy_host = '184.174.24.237'  
proxy_port = '6813'              
proxy_username = 'fjhdetvm'  
proxy_password = 'dj626ier4ofb'  

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

page_url = 'https://mall.elady.com/items/category/BAGS/?limit=120'

proxy_url = f'http://{proxy_username}:{proxy_password}@{proxy_host}:{proxy_port}'

current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
csv_file = f'elady_{current_time}.csv'

def product_info(product_url, retry = 3):
    if retry ==0:
        return None
    try:
        if not product_url.startswith('http'):
            raise ValueError("Invalid product URL")
        
        response = requests.get(product_url, proxies={'http': proxy_url, 'https': proxy_url}, headers=headers,auth=HTTPProxyAuth(proxy_username, proxy_password))
        if response.status_code != 200:
            raise Exception(f"Failed to retrieve {product_url}, status code: {response.status_code}")
            
        soup = BeautifulSoup(response.content, 'html.parser')
        
        product_name = soup.find('div',class_='detail__name').text.strip()

        product_price = soup.find('span',class_='price').text.strip()

        sku = soup.find('p',class_='detail__code').text.replace('SKU:','').strip()

        main_image = soup.find('img',id='image-main').get('src')

        brand, line, product_type, material, color, closure, inner_pocket, gender, size, handle_length, strap_length = None, None, None, None, None, None, None, None, None, None, None 
        all_x_td = soup.find('div', id = 'detail_param_content1').find_all('td', class_='y')
        all_x_td = [i for i in all_x_td if i.text != '']
        all_y_td = soup.find('div', id = 'detail_param_content1').find_all('td', class_='x')
        for num in range(len(all_x_td)):
            key = all_x_td[num].text
            values = all_y_td[num].find_all('tr')
            if len(values)>1:
                value = ', '.join([i.text.strip() for i in values if hasattr(i, 'text') and i.text != '']).strip()
            else:
                value = values[0].text.strip()

            if key == 'Brand':
                brand = value
            elif key == 'Line':
                line = value
            elif key == 'Model':
                model = value
            elif key == 'Country of Origin':
                country_of_origin = value
            elif key == 'Type':
                product_type = value
            elif key == 'Material':
                material = value
            elif key == 'Leather/Fur Type':
                leather_or_fur_type = value
            elif key == 'Color':
                color = value
            elif key == 'Closure':
                closure = value
            elif key == 'Inner Pocket':
                inner_pocket = value
            elif key == 'Gender':
                gender = value
            elif key == 'Size (HxWxD)':
                size = value
            elif key == 'Handle Length':
                handle_length = value
            elif key == 'Strap Length':
                strap_length = value

        all_x_td = soup.find('div', id = 'detail_param_content2').find_all('td', class_='y')
        all_x_td = [i for i in all_x_td if i.text != '']
        all_y_td = soup.find('div', id = 'detail_param_content2').find_all('td', class_='x')

        accessories, condition = None, None
        for num in range(len(all_x_td)):
            key = all_x_td[num].text
            values = all_y_td[num].find_all('tr')
            if len(values)>1:
                value = ', '.join([i.text.strip() for i in values if hasattr(i, 'text') and i.text != '']).strip()
            else:
                value = values[0].text.strip()

            if key == 'Accessories':
                accessories = value
            elif key == 'Condition':
                condition = value

        di = {'product_name':product_name, 'product_price':product_price, 'sku':sku, 'main_image':main_image,
         'brand':brand, 'line':line, 'model':model, 'country_of_origin':country_of_origin, 'product_type':product_type, 'material':material, 
              'leather_or_fur_type':leather_or_fur_type, 'color':color, 'closure':closure,
          'inner_pocket':inner_pocket, 'gender':gender, 'size':size, 'handle_length':handle_length, 'strap_length':strap_length,
             'accessories':accessories, 'condition':condition}
        return di
    
    except Exception as e:
        print(f"Error in product_info for {product_url}: {e}")
        print('Retrying this URL again!!, Remaining retry: ',retry)
        return product_info(product_url, retry-1)
        
    

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    
    fieldnames = ['product_name', 'product_price', 'sku', 'main_image', 'brand', 'line', 'model', 'country_of_origin', 'product_type', 'material', 
                  'leather_or_fur_type', 'color',
                  'closure', 'inner_pocket', 'gender', 'size', 'handle_length', 'strap_length', 'accessories', 'condition']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    
    while True:
        try:
            response = requests.get(page_url, proxies={'http': proxy_url, 'https': proxy_url}, headers=headers,auth=HTTPProxyAuth(proxy_username, proxy_password))
            if response.status_code != 200:
                raise Exception(f"Failed to retrieve page, status code: {response.status_code}")

            soup = BeautifulSoup(response.content, 'html.parser')
            try:
                page_url = 'https://mall.elady.com'+soup.find('a', title='Next').get('href')
            except:
                page_url = None

            urls = soup.find_all('a', class_='item_archive__link')

            for url in urls:
                product_url = 'https://mall.elady.com' + url.get('href')
                print(product_url)
                product_data = product_info(product_url)
                
                if product_data:
                    writer.writerow(product_data)
                    file.flush()  
                    
                length = sum(1 for _ in open(csv_file)) - 1  # Subtract 1 to exclude the header
                print(f"Current CSV length (excluding header): {length}")
                
            if page_url == None:
                break
        except Exception as e:
            print(f"Error while processing page: {e}")
            break

NameError: name 'datetime' is not defined